<a href="https://colab.research.google.com/github/parulnith/GenAI/blob/main/AI_card_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 AI-Powered Greeting Card Generator

## 📌 About This Notebook
This notebook demonstrates how to **create an AI-powered greeting card generator** using **Gradio** and **Google Gemini AI**.

You will be able to:
- Generate **custom greeting cards** with a message.
- Use **Google Gemini AI** to create **aesthetic card designs**.
- Deploy a **Gradio web app** in **Colab/Jupyter Notebook**.






# Steps to Use This Notebook
* Run all the cells in this notebook.
* Enter your Google Gemini API Key in the designated section.
* Click the Gradio link (which will be generated) to access your web app.
* Enter details (like occasion, sender name, and recipient name).
* Click "Generate Greeting Card" to see the AI-created card!
* Click "Generate a New Card" to reset the fields.







## 📌 Step 1: Install Required Libraries
Run the following cell to **install Gradio and Google Gemini AI**:

In [30]:
# Install necessary libraries
!pip install gradio google-genai --quiet


# 🔑 Step 1: Set Up Google Gemini API Key
Google Gemini AI requires an API key to generate images. Follow these steps:

* Go to Google AI Studio → https://aistudio.google.com
* Sign in with your Google account and get an API key.
* Use the secrets manager to securely store your API key:


In [31]:

from google.colab import userdata

# Set API key securely
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")


# Step 3: Import Necessary Libraries
## Run the following cell to import required libraries:

In [32]:
import os
import gradio as gr
from google import genai
from google.genai import types


# Ensure 'static' directory exists to store generated images
output_dir = "/content/static"
os.makedirs(output_dir, exist_ok=True)



# Step 4: Define Helper Functions
These functions handle image generation using Google Gemini AI.

In [33]:
# Function to save binary image files
def save_binary_file(file_name, data):
    with open(file_name, "wb") as f:
        f.write(data)

# Function to generate a greeting card using Google's Gemini AI
def generate(occasion, sender_name, recipient_name, custom_message):
    # Initialize Google Gemini AI client
    client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))

    # Specify the AI model
    model = "gemini-2.0-flash-exp-image-generation"

    # Generate a default message if no custom message is provided
    if not custom_message:
        custom_message = f"Happy {occasion}, {recipient_name}! With love, {sender_name}."
    else:
        custom_message = f"Dear {recipient_name},\n\n{custom_message}\n\nFrom, {sender_name}."

    # Request AI to generate an image based on the given message
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text=f"Generate a greeting card with a soft background and clear text that says:\n\n{custom_message}"
                ),
            ],
        ),
    ]

    # AI response configuration
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_modalities=["text", "image"],
    )

    # Request the AI to generate content
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )

    # Extract the generated image and save it
    for candidate in response.candidates:
        for part in candidate.content.parts:
            if part.inline_data:
                file_name = f"static/{occasion.lower()}_card.png"
                save_binary_file(file_name, part.inline_data.data)
                return file_name  # Return the image file path
    return None  # Return None if image generation fails


# Step 5: Create a Gradio Interface
This creates a user-friendly web app to interact with the AI.

In [40]:
# Function to interface Gradio with our AI model
def gradio_interface(occasion, sender_name, recipient_name, custom_message):
    image_path = generate(occasion, sender_name, recipient_name, custom_message)
    if image_path:
        return image_path  # Returning the image file path
    else:
        return None  # Return None if image generation fails

# Create a Gradio app with a simple user interface
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("<center><h1>🎨 Greeting Card Generator 🎨</h1></center>")  # App Title

    with gr.Row():
        with gr.Column():
            # Dropdown for selecting the occasion
            occasion = gr.Dropdown(
                label="Choose an Occasion",
                choices=["Birthday", "Christmas", "New Year", "Anniversary", "Others"],
                value="Birthday"
            )
            # Input fields for sender's and recipient's name
            sender_name = gr.Textbox(label="Sender's Name")
            recipient_name = gr.Textbox(label="Recipient's Name")
            # Custom message input
            custom_message = gr.Textbox(label="Custom Message (Leave blank for auto-generated)")
            # Buttons for generating and resetting
            generate_button = gr.Button("Generate Greeting Card")
            reset_button = gr.Button("Generate a New Card")

        with gr.Column():
            output_image = gr.Image(label="Generated Greeting Card",width=600, height=550)

    # When the Generate button is clicked, run the `gradio_interface` function
    generate_button.click(
        fn=gradio_interface,
        inputs=[occasion, sender_name, recipient_name, custom_message],
        outputs=output_image
    )

    # When the Reset button is clicked, clear all input fields
    reset_button.click(
        fn=lambda: ("Birthday", "", "", ""),
        inputs=[],
        outputs=[occasion, sender_name, recipient_name, custom_message]
    )




# Step 6: Run the Gradio App
This command launches the Gradio web app and provides a public link.

In [41]:
# Launch the Gradio app with sharing enabled.
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://130050c8bbdc92a216.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
